In [ ]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import VGG16



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = ('/content/drive/MyDrive/dataset25s/train')
val_dir = ('/content/drive/MyDrive/dataset25s/val')
test_dir = ('/content/drive/MyDrive/dataset25s/test')

In [ ]:
IMAGE_SIZE = 244
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
) 

valid_datagen = ImageDataGenerator(
        rescale=1./255,
       )


train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size =(IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb'
)

valid_gen = valid_datagen.flow_from_directory(
    val_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    color_mode= "rgb"
)
    


In [ ]:
 base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")


In [ ]:
for layer in base_model.layers[:17]:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)



In [ ]:
model = tf.keras.Model(base_model.input, x)


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])


In [ ]:
from matplotlib import pyplot as plt

history = model.fit(train_gen,epochs = 40,validation_data = valid_gen)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(
        rescale=1./255)


test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical'
)


pred = model.predict_generator(test_gen, steps=10, verbose=1)

In [ ]:
final_loss, final_acc = model.evaluate(valid_gen, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))